In [75]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqFeature import FeatureLocation
import sqlite3
import gffutils

In [115]:
gene2bb = '/Users/mf019/bioinformatics/longread_pangenome/group2bb/output/lr_groupID_to_BBgene.tsv'
gene_db = '/Users/mf019/bioinformatics/longread_pangenome/notebooks/metadb/Bbss_db_v2.db'
fixed_out = 'bb_rs_gene_coords_b31_v1.tsv'
b31_genbank = '/Users/mf019/bioinformatics/longread_pangenome/ref/renamed_GCF_000008685.2.gbff'
b31_gff = '/Users/mf019/bioinformatics/longread_pangenome/ref/GCF_000008685.2_ASM868v2_genomic.gff'

In [81]:
g2bb_df = pd.read_csv(gene2bb, sep='\t')
g2bb_df.head()

,ID,group,gene,percent_ident,alignment_length,E-score
0,DMPBEC_00005,Uncharacterized protein BB_0001,gene-BB_RS00005,100.000,564,0.0
1,DMPBEC_00010,Glycoside hydrolase family 3 N-terminal domain...,gene-BB_RS00010,100.000,1029,0.0
2,DMPBEC_00015,group_1482,gene-BB_RS00015,99.523,1467,0.0
3,DMPBEC_00020,group_3841,gene-BB_RS00020,99.331,1794,0.0
4,DMPBEC_00025,trpS,gene-BB_RS00025,99.716,1056,0.0


In [57]:
for record in SeqIO.parse(b31_genbank, 'genbank'):
    print(record.id)
    replicon = record.id
    for feature in record.features:
        if feature.type == 'CDS':
            name = feature
            print(feature.qualifiers)
            break
            locus_id = feature.qualifiers['locus_tag'][0]
            start = feature.location.start
            end = feature.location.end
            print(name, locus_id, replicon, start, end)

chromosome
{'locus_tag': ['BB_0001'], 'codon_start': ['1'], 'transl_table': ['11'], 'product': ['conserved hypothetical protein'], 'protein_id': ['AAC66406.1'], 'translation': ['MILMKYSAILLICSVNLFCFQNKLTTSRWEFPKEDLIKKKIKIGIIYHNYINSIFYNENYKYIAFIGILTSYNEWIEIQFSPINFFTIPTNKDFISNTYFNLAFTIYITKYSILTDTLAIKFFIGTQIDLTLRTTIFTGKTTHAFLYPILPIITFKFEIDFIPNNYSIYYKLSTSFKEFILLDLGISIFI']}
lp5
{'locus_tag': ['BB_T01'], 'codon_start': ['1'], 'transl_table': ['11'], 'product': ['conserved hypothetical protein'], 'protein_id': ['AAF08550.2'], 'translation': ['MYKAIKEQQEIEIDHACRILILTATIFEINSIFENYYQKTLLKKYNENLKNKNLPPSNISTMKKYLNQLEKEIKIIAKFYFKNDQSLIYCKLNYTLEKICLKLIKFYKKFYKELKQFTQKNITT']}
cp9
{'locus_tag': ['BB_C01'], 'note': ['identified by match to protein family HMM PF02414'], 'codon_start': ['1'], 'transl_table': ['11'], 'product': ['BBC01'], 'protein_id': ['AAC66310.1'], 'translation': ['MENSKKNTLCQNKTQHKSIVLISTLEYINKTHNKYTQKNILYYFNENLKRNGQLPVKIKTLQNYLYKLEKEIKVTTNYHKHLGVNCGTEIYYQLNFSKKECYQKIYKYFQEKKDLRFQNRAT

In [78]:
db = gffutils.create_db(b31_gff, dbfn='test.db', force=True, keep_order=True, merge_strategy='merge', sort_attribute_values=True)


In [95]:
RSid2name = {"NC_001318.1": 'chromosome',
"NC_000957.1": 'lp5',
"NC_001904.1": 'cp9',
"NC_001849.2": 'lp17',
"NC_000955.2": 'lp21',
"NC_001850.1": 'lp25',
"NC_001903.1": 'cp26',
"NC_001851.2": 'lp28-1',
"NC_001852.1": 'lp28-2',
"NC_001853.1": 'lp28-3',
"NC_001854.1": 'lp28-4',
"NC_000948.1": 'cp32-1',
"NC_000949.1": 'cp32-3',
"NC_000950.1": 'cp32-4',
"NC_000951.1": 'cp32-6',
"NC_000952.1": 'cp32-7',
"NC_000953.1": 'cp32-8',
"NC_000954.1": 'cp32-9',
"NC_001855.1": 'lp36',
"NC_001856.1": 'lp38',
"NC_001857.2": 'lp54',
"NC_000956.1": 'lp56',}

In [106]:

header = 'gene\treplicon\tstart\tend\n'
rows = [header]
#rows.append(header)
for idx, row in g2bb_df.iterrows():
    gene_id = row['gene']
    gene = db[row['gene']]
    replicon = RSid2name[gene.chrom]
    name = gene['Name'][0]
    start = gene.start
    stop = gene.end
    #print(gene_id, replicon, start, stop)
    rows.append(f'{gene_id}\t{replicon}\t{start}\t{stop}\n')

In [108]:
with open(fixed_out, 'w') as f:
    for line in rows:
        f.write(line)


In [110]:

fixed_df = pd.read_csv(fixed_out, sep='\t')


In [114]:
fixed_df

,gene,replicon,start,end
0,gene-BB_RS00005,chromosome,105,677
1,gene-BB_RS00010,chromosome,768,1796
2,gene-BB_RS00015,chromosome,1784,3249
3,gene-BB_RS00020,chromosome,3395,5188
4,gene-BB_RS00025,chromosome,5251,6306
...,...,...,...,...
23429,gene-BB_RS05825,lp28-1,18870,19979
23430,gene-BB_RS05830,lp28-1,20027,21193
23431,gene-BB_RS05820,lp28-1,18427,18822
23432,gene-BB_RS04830,lp38,38349,38522


In [113]:
g2bb_df

,ID,group,gene,percent_ident,alignment_length,E-score
0,DMPBEC_00005,Uncharacterized protein BB_0001,gene-BB_RS00005,100.000,564,0.000000e+00
1,DMPBEC_00010,Glycoside hydrolase family 3 N-terminal domain...,gene-BB_RS00010,100.000,1029,0.000000e+00
2,DMPBEC_00015,group_1482,gene-BB_RS00015,99.523,1467,0.000000e+00
3,DMPBEC_00020,group_3841,gene-BB_RS00020,99.331,1794,0.000000e+00
4,DMPBEC_00025,trpS,gene-BB_RS00025,99.716,1056,0.000000e+00
...,...,...,...,...,...,...
23429,JOOONJ_06265,group_2488,gene-BB_RS05825,70.697,488,2.740000e-55
23430,JOOONJ_06265,group_2488,gene-BB_RS05830,70.880,443,6.030000e-51
23431,JOOONJ_06265,group_2488,gene-BB_RS05820,69.186,344,1.620000e-26
23432,JOOONJ_06265,group_2488,gene-BB_RS04830,83.333,66,7.360000e-12


,ID,group,gene,percent_ident,alignment_length,E-score,replicon,start,end
0,DMPBEC_00005,Uncharacterized protein BB_0001,gene-BB_RS00005,100.000,564,0.000000e+00,chromosome,105,677
1,DMPBEC_00010,Glycoside hydrolase family 3 N-terminal domain...,gene-BB_RS00010,100.000,1029,0.000000e+00,chromosome,768,1796
2,DMPBEC_00010,Glycoside hydrolase family 3 N-terminal domain...,gene-BB_RS00010,100.000,1029,0.000000e+00,chromosome,768,1796
3,DMPBEC_00015,group_1482,gene-BB_RS00015,99.523,1467,0.000000e+00,chromosome,1784,3249
4,DMPBEC_00020,group_3841,gene-BB_RS00020,99.331,1794,0.000000e+00,chromosome,3395,5188
...,...,...,...,...,...,...,...,...,...
1217843,JOOONJ_06265,group_2488,gene-BB_RS04820,83.607,61,8.970000e-11,lp38,37415,37774
1217844,JOOONJ_06265,group_2488,gene-BB_RS04820,83.607,61,8.970000e-11,lp38,37415,37774
1217845,JOOONJ_06265,group_2488,gene-BB_RS04820,83.607,61,8.970000e-11,lp38,37415,37774
1217846,JOOONJ_06265,group_2488,gene-BB_RS04820,83.607,61,8.970000e-11,lp38,37415,37774
